## 그래프에 쓰레드 수준의 영속성을 추가하는 방법
많은 AI 애플리케이션은 여러 상호작용에서 컨텍스트를 공유하기 위해 메모리를 필요로 함

LangGraph에서는 쓰레드 수준의 영속성을 사용하여 이러한 메모리를 모든 StateGraph에 추가할 수 있음
- LangGraph 그래프를 생성할 때(그래프를 컴파일할 때) 체크포인터를 추가하여 그래프의 상태를 유지하도록 설정할 수 있음

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 그래프 정의
채팅 모델을 호출하는 단일 노드 그래프를 사용

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, MessagesState, START


def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
graph = builder.compile()

이 그래프를 사용하려고 하면 대화의 맥락이 상호 작용 전반에 걸쳐 유지되지 않습니다.

In [4]:
input_message = {"role": "user", "content": "hi! I'm bob"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

input_message = {"role": "user", "content": "what's my name?"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hi Bob! How can I assist you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about users unless it has been shared with me in the course of our conversation. How can I assist you today?


### 영속성 추가
영속성을 추가하려면 그래프를 컴파일 할 때 체크포인터를 전달해야 합니다.

In [5]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
# LangGraph Cloud나 LangGraph Studio를 사용하는 경우, 그래프를 컴파일할 때 checkpointer를 따로 전달할 필요가 없습니다.(자동으로 처리되기 때문입니다.)

In [6]:
config = {"configurable": {"thread_id": "1"}}
input_message = {"role": "user", "content": "hi! I'm bob"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hi Bob! How can I help you today?


In [7]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob! What would you like to talk about?


새 대화를 시작하고 싶다면, 다른 thread_id를 전달하면 됩니다.

In [8]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in graph.stream(
    {"messages": [input_message]},
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about users unless you provide it during our conversation. If you'd like me to address you by a specific name, please let me know!
